In [1]:
import os
import pandas as pd
import torch
from lib.datasets import get_stock_price,sample_indices,train_test_split
from lib.aug import apply_augmentations,parse_augmentations
from typing import List
from torch import nn
from typing import List
import torch.nn.functional as F

In [2]:
data_config = {
    "ticker" : "^GSPC",
    "interval" : "1d",
    "column" : 1,  
    "window_size" : 20,
    "dir" : "datasets",
    "subdir" : "stock"
}
sig_config = {
    "augmentations": [
        {"name": "AddTime"},
        {"name": "LeadLag"},
    ],
    "device" : "cuda:0",
    "depth" : 4,
}

In [3]:
tensor_data = get_stock_price(data_config)
x_real_train, x_real_test = train_test_split(tensor_data, train_test_ratio=0.8, device=sig_config["device"])
if sig_config["augmentations"] is not None:
    sig_config["augmentations"] = parse_augmentations(sig_config.get('augmentations'))
print("Before augmentation shape:",x_real_train.shape)
if sig_config["augmentations"] is not None:
    # Print the tensor shape after each augmentation
    x_aug_sig = apply_augmentations(x_real_train,sig_config["augmentations"])
    # Input dimension of encoder
    # We'll flat the tensor
    input_dim = x_aug_sig.shape[1]*x_aug_sig.shape[2]
print("After augmentation shape:",x_aug_sig.shape)
x_aug_sig = x_aug_sig.to(sig_config["device"])

Rolled data for training, shape torch.Size([1232, 20, 1])
Before augmentation shape: torch.Size([985, 20, 1])
torch.Size([985, 20, 2])
torch.Size([985, 39, 4])
After augmentation shape: torch.Size([985, 39, 4])


In [4]:
class VAE(nn.Module):
    def __init__(self, x_aug_sig, epoch, batch_size, hidden_dims: List, device) -> None:
        super(VAE, self).__init__()

        self.x_aug_sig = x_aug_sig
        print("Input tensor shape: {}".format(x_aug_sig.shape))
        self.epoch = epoch
        self.batch_size = batch_size
        self.device = device

        # Assume len(hidden_dims)=3.
        self.encoder_mu = nn.Sequential(
            nn.Linear(hidden_dims[0],hidden_dims[1]),
            nn.LeakyReLU(),
            nn.Linear(hidden_dims[1],hidden_dims[2]),
            nn.LeakyReLU(),
        )
        self.encoder_sigma = nn.Sequential(
            nn.Linear(hidden_dims[0],hidden_dims[1]),
            nn.Tanh(),
            nn.Linear(hidden_dims[1],hidden_dims[2]),
            nn.LeakyReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dims[2],hidden_dims[1]),
            nn.LeakyReLU(),
            nn.Linear(hidden_dims[1],hidden_dims[0]),
        )

        # To device
        self.encoder_mu.to(device)
        self.encoder_sigma.to(device)
        self.decoder.to(device)
    
    def encode(self, x):
        x_flatten = x.view(self.batch_size,-1)
        mean = self.encoder_mu(x_flatten)
        log_var = self.encoder_sigma(x_flatten)
        # Clipping
        log_var = torch.clamp(log_var, min=-10, max=10)
        noise = torch.randn(self.batch_size,mean.shape[1]).to(self.device)
        z = mean + torch.exp(0.5*log_var).mul(noise)
        return mean, log_var, z
        
    def decode(self,z):
        reconstructed_data = self.decoder(z)
        return reconstructed_data

    def loss(self,mean,log_var,sample_data,reconstructed_data):
        # Reconstruction loss 
        recon_loss = F.mse_loss(sample_data, reconstructed_data, reduction='mean')
        # print(recon_loss.item())
        # KL divergence
        kl_loss = 0.5 * ((mean.pow(2) + log_var.exp() - 1 - log_var).mean()).sum()
        # Total VAE loss
        loss = recon_loss + kl_loss
        return loss
    
def train(model,optimizer):
    early_stop = 500
    cnt = 0
    min_loss = float('inf')
    for i in range(model.epoch):
        # Sample time indices of size equal to the batch size
        # From sefl.x_aug_sig
        time_indics = sample_indices(model.x_aug_sig.shape[0],model.batch_size,"cuda")
        sample_data = model.x_aug_sig[time_indics]
        # Encode 
        mean, log_var, z = model.encode(sample_data)
        # Decode
        reconstructed_data = model.decode(z)
        # Calculate loss
        loss = model.loss(mean,log_var,sample_data.view(model.batch_size,-1),reconstructed_data)
        # Backpropogation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Print loss
        if i%500==0:
            print("Epoch {} loss {}".format(i,loss.item()))
        # Early stop
        if loss.item()<min_loss:
            min_loss = loss.item()
            cnt = 0
        else:
            cnt += 1
            if cnt>early_stop:
                break


In [5]:
lr = 1e-4
batch_size = 200
epoch = 20001
hidden_dims = [input_dim,12,3]
VAE = VAE(x_aug_sig=x_aug_sig,epoch=epoch,batch_size=batch_size,hidden_dims=hidden_dims,device='cuda')
print(VAE)
optimizer = torch.optim.Adam(VAE.parameters(),lr=lr)

Input tensor shape: torch.Size([985, 39, 4])
VAE(
  (encoder_mu): Sequential(
    (0): Linear(in_features=156, out_features=12, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=12, out_features=3, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  )
  (encoder_sigma): Sequential(
    (0): Linear(in_features=156, out_features=12, bias=True)
    (1): Tanh()
    (2): Linear(in_features=12, out_features=3, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=12, out_features=156, bias=True)
  )
)


After add a line ``log_var = torch.clamp(log_var, min=-10, max=10)``, the loss is no longer ``nan``. 

In [6]:
train(VAE,optimizer=optimizer)

Epoch 0 loss 9686226.0
Epoch 500 loss 9070517.0
Epoch 1000 loss 6220897.0
Epoch 1500 loss 1736919.5
Epoch 2000 loss 1202789.875
Epoch 2500 loss 930679.0625
Epoch 3000 loss 741196.25
Epoch 3500 loss 614923.6875
Epoch 4000 loss 490475.0625
Epoch 4500 loss 430703.84375
Epoch 5000 loss 359672.21875
Epoch 5500 loss 289123.28125
Epoch 6000 loss 264180.875
Epoch 6500 loss 215572.90625
Epoch 7000 loss 190932.40625
Epoch 7500 loss 163359.484375
Epoch 8000 loss 138938.828125
Epoch 8500 loss 120657.2578125
Epoch 9000 loss 106078.2421875
Epoch 9500 loss 94310.390625
Epoch 10000 loss 80251.2265625
Epoch 10500 loss 73686.34375
Epoch 11000 loss 64317.55078125
Epoch 11500 loss 54875.4921875
Epoch 12000 loss 48903.71875
Epoch 12500 loss 45501.37109375
Epoch 13000 loss 39819.52734375
Epoch 13500 loss 36470.34765625
Epoch 14000 loss 31373.701171875
Epoch 14500 loss 29083.810546875
Epoch 15000 loss 25622.19921875
Epoch 15500 loss 23948.1796875
Epoch 16000 loss 21517.134765625
Epoch 16500 loss 19108.367187